In [6]:
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import datetime
import MetaTrader5 as mt5

In [8]:
class BackTest:
    
    def __init__(self, instrument):
        
#         self.df = pd.DataFrame(yf.download(tickers=instrument, start = '2022-10-01', end = '2023-06-01', interval='1h'))
        if not mt5.initialize(login=114781990, server="Exness-MT5Trial6",password="Mypassword$1234"):
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        login = 114781990
        password = 'Mypassword$1234'
        server = 'Exness-MT5Trial6'

        mt5.login(login, password, server)
        print('Working on: ', instrument[0])
        rate = mt5.copy_rates_from(instrument[0], mt5.TIMEFRAME_M15, datetime.datetime.now(), instrument[2])
        self.df = pd.DataFrame(rate)
        
         
        self.df['date']=pd.to_datetime(self.df['time'], unit='s')

        
        self.analyse = pd.DataFrame(columns=['TOTAL TRADES', 'WINS', 'LOSSES', 'PERFORMANCE', 'WIN RATE', 
                                'WINNING STREAK', 'LOSING STREAK', 'MAX DRAWDOWN', 'AVG GAIN', 'AVG LOSS'])
        
        '''Renaming columns and deleting unnecessary columns'''

        self.df.set_index(np.arange(len(self.df)), inplace = True)
        self.df = self.df.drop(['spread', 'real_volume', 'tick_volume'], axis='columns')
        
        
        self.one_pip_value = instrument[1]
        self.account = 10000
#         print(self.df.head())
        
    def display(self):
        
#         print(self.df.head())
#         self.df.to_excel('df.xlsx')
#         self.squeeze.to_excel('squeeze.xlsx')
        print(self.analyse)
        self.result.to_excel('EURUSD.xlsx')
#         return(self.analyse)
    
    
    def in_squeeze(self, df):
        return df['lower_band'] > df['lower_keltner'] and df['upper_band'] < df['upper_keltner']
    
    
    def define_trend(self, current_high, current_low, prev_high, prev_low, days, i):
        #       Uptrend
            if current_high > prev_high and current_low > prev_low:
    #           Find point at which current sample
    #           crossed prev high

                stream = self.df.loc[self.df.index[i : i+days], 'high']
                up_candle_index = stream.loc[stream > prev_high].index[0]
                
    #             if df.loc[i+15, 'high'] > prev_high:
                    
                self.df.loc[up_candle_index : i+days, 'trend'] = 'up'

            
                if self.df.iloc[i-1]['trend'] == 'up':
                    self.df.loc[i: up_candle_index, 'trend'] = 'up'

                else:
                    self.df.loc[i: up_candle_index, 'trend'] = 'no trend'


    #         Downtrend
            elif (current_high < prev_high) and (current_low < prev_low):
    #           Find point at which current sample 
    #           crossed prev low

                stream = self.df.loc[self.df.index[i : i+days], 'low']
                down_candle_index = stream.loc[stream < prev_low,].index[0]
               
#                 print('DownCandleIndex: ', down_candle_index)
#                 print('Days Callee: ', days)
                
                self.df.loc[down_candle_index : i+days, 'trend'] = 'down'

                if self.df.iloc[i-1]['trend'] == 'down':
                    self.df.loc[i: down_candle_index, 'trend'] = 'down'

                else:
                    self.df.loc[i: down_candle_index, 'trend'] = 'no trend'


                    
            else:
                self.df.loc[i : i+days, 'trend'] = 'no trend'

    
    def get_trend(self, i=0):
        
        length = len(self.df)

        # print('Length: ', length)
        
        days = 21

        prev_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
        
        prev_low = self.df.loc[self.df.index[i : i+days], 'low'].min()

        while(i + days  < length):

            # print('In Trend !!')
        
            i += days
            
            current_high = self.df.loc[self.df.index[i : i+days], 'high'].max()
            current_low = self.df.loc[self.df.index[i : i+days], 'low'].min()
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            prev_high = current_high
            prev_low = current_low
            
                    
                
        if i < length-1:
            prev_high = self.df.loc[self.df.index[i-days:i], 'high'].max()
            prev_low = self.df.loc[self.df.index[i-days:i], 'low'].min()
            
            current_high = self.df.loc[self.df.index[i : length], 'high'].max()
            current_low = self.df.loc[self.df.index[i : length], 'low'].min()
            
            days = length-i
            
            self.define_trend(current_high, current_low, prev_high, prev_low, days, i)
            
            
        

    # KELTNER CHANNEL CALCULATION

    def get_kc(self, high, low, close, kc_lookback, multiplier, atr_lookback):
        tr1 = pd.DataFrame(high - low)
        tr2 = pd.DataFrame(abs(high - close.shift()))
        tr3 = pd.DataFrame(abs(low - close.shift()))
        
        # Calculate True Range as the element-wise maximum of the three components
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        atr = tr.ewm(alpha=1/atr_lookback).mean()

        kc_middle = close.ewm(span=kc_lookback).mean()
        kc_upper = kc_middle + multiplier * atr
        kc_lower = kc_middle - multiplier * atr

        return kc_middle, kc_upper, kc_lower
    
    
    def indicators(self):
        '''Adding EMAs, BB, Keltner Channel and RSI'''
        
        self.df['EMA_8'] = talib.EMA(self.df['close'], timeperiod=8)

        # EMA 20 group
#         self.df['EMA_15'] = talib.EMA(self.df['close'], timeperiod=15)
        self.df['EMA_20'] = talib.EMA(self.df['close'], timeperiod=20)
#         self.df['EMA_22'] = talib.EMA(self.df['close'], timeperiod=22)
        # EMA 50 group
#         self.df['EMA_48'] = talib.EMA(self.df['close'], timeperiod=48)
#         self.df['EMA_50'] = talib.EMA(self.df['close'], timeperiod=50)
#         self.df['EMA_52'] = talib.EMA(self.df['close'], timeperiod=52)
        # EMA 200 group
#         self.df['EMA_198'] = talib.EMA(self.df['close'], timeperiod=198)
        self.df['EMA_200'] = talib.EMA(self.df['close'], timeperiod=200)
#         self.df['EMA_202'] = talib.EMA(self.df['close'], timeperiod=202)

        self.df['RSI'] = talib.RSI(self.df['close'], timeperiod=21)
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)
        # Identify trend on 21 days highs & lows
        self.get_trend()
#         self.df.to_excel('check_trend.xlsx')

    #     df.head()

        '''Bollinger Band'''
        self.df['20sma'] = self.df['close'].rolling(window=20).mean()
        self.df['stddev'] = self.df['close'].rolling(window=20).std()
        self.df['lower_band'] = self.df['20sma'] - (2 * self.df['stddev'])
        self.df['upper_band'] = self.df['20sma'] + (2 * self.df['stddev'])

        '''Keltners Channel'''
        self.df['TR'] = abs(self.df['high'] - self.df['low'])
        self.df['ATR'] = self.df['TR'].rolling(window=20).mean()

        self.df['lower_keltner'] = self.df['20sma'] - (self.df['ATR'] * 1.2)
        self.df['upper_keltner'] = self.df['20sma'] + (self.df['ATR'] * 1.2)
        
        
        self.df['squeeze_on'] = self.df.apply(self.in_squeeze, axis=1)

        self.df = self.df.drop(['stddev', 'lower_band', 'upper_band', '20sma', 'TR', 'ATR', 'lower_keltner', 'upper_keltner'],axis=1)

        '''Get Keltner for trading'''
        
        self.df['EMA_20'],self.df['upper_keltner'],self.df['lower_keltner'] = self.get_kc(self.df['high'], self.df['low'], self.df['close'], kc_lookback = 20, multiplier = 2.25, atr_lookback = 10)
        
        self.df = self.df.dropna()

        self.df.set_index(np.arange(len(self.df)), inplace = True)

        self.df.index.name='sn'
        # print('Updated DF: ',self.df.head())
        
#         self.df.to_excel('check_trend.xlsx')
        
    
    def candle(self, df):
        '''Return candle colour'''

        if df['close'] - df['open'] > 0:
            return 'G'

        else:
            return 'R'
    

    def check_body(self, df):
        #     Calculate 0.7 % of entry price
    #     fOR 1H FOREX
            one_percent = df['open']/100 
            percent = one_percent * 0.06
            hammer_per = one_percent * 0.06


        #     Calculate candle size close-open
            if df['candle'] == 'G':
                candle_size = df['close'] - df['open']
                upper_wick = df['high'] - df['close']
                lower_wick = df['open'] - df['low']

            else:
                candle_size = abs(df['close'] - df['open'])
                upper_wick = df['high'] - df['open']
                lower_wick = df['close'] - df['low']

            upper_wick_per = (upper_wick/one_percent) 
            lower_wick_per = (lower_wick/one_percent) 



        #     check for entry candle size and Max candle size 4.5 %
            if ((candle_size >= percent) and candle_size <= (4.5 * one_percent) and
             (candle_size > upper_wick) and (candle_size > lower_wick)) and (upper_wick_per < 0.06) and (lower_wick_per < 0.06):
                
                if df['candle'] == 'G' and df['high'] < df['upper_keltner'] :

                    return 1
                
                elif df['candle'] == 'R' and df['low'] > df['lower_keltner'] :

                    return 1
                
                else:
                    
                    return 0

        #     condition for hammmer
        #     lower wick is atleast 50 % of total size 
        #     Green Hammer 
        # need to add max wick of opposite dir
            elif ( (df['candle'] == 'G') and (df['open'] - df['low']) >= ((df['high'] - df['low']) * 0.5) and
                  (candle_size >= hammer_per) and (candle_size > upper_wick)and
                  candle_size <= (4.5 * one_percent) and ((upper_wick_per < 0.06))  
                 and df['high'] < df['upper_keltner']):

                return 1


    #     For Red Hammer
    # need to add max wick of opposite dir
            elif ( (df['candle'] == 'R') and (df['high'] - df['open']) >= (abs(df['high'] - df['low']) * 0.5) and 
                  (candle_size >= hammer_per) and (candle_size > lower_wick)and candle_size <= (4.5 * one_percent) and 
                  ((lower_wick_per < 0.06)) and df['low'] > df['lower_keltner'] ):

                return 1


            else:
                return 0

        
    
    def check_prev_candle(self, element):

        current_index = self.df.loc[self.df.date ==  element.date].index[0]
        current_close = element.close
        current_candle = element.candle



        if current_index > 0:
            prev_index = current_index - 1 
#           prev candle colour
            prev_candle = self.df.iloc[prev_index]['candle']

#           current candle colour
            current_candle = self.df.iloc[current_index]['candle']
            
            if current_candle == 'G':
#                 Both green current close must be higher
                if prev_candle == 'G' and self.df.iloc[current_index]['close'] > self.df.iloc[prev_index][ 'close']:
                    return 1
                
#                If last candle Red then entry candle must close above last candle open
                elif prev_candle == 'R' and self.df.iloc[current_index]['close'] > self.df.iloc[prev_index][ 'open']:
                    return 1

                else:
                    return 0
                
            elif current_candle == 'R':
#                 Both red current close must be lower
                if prev_candle == 'R' and self.df.iloc[current_index]['close'] < self.df.iloc[prev_index][ 'close']:
                    return 1
                
#                If last candle green then entry candle must close below last candle open
                elif prev_candle == 'G' and self.df.iloc[current_index]['close'] < self.df.iloc[prev_index][ 'open']:
                    return 1

                else:
                    return 0
                
    #     How to check for the 1st candle?
        return 0
    
    
#   SL Method

    def set_sl(self, df):
        '''Set SL 1% above/below to entry candle'''
        
        if df['pos_trade'] == '1':
            
            candle_size = df['high'] - df['low']
            
            if df['candle'] == 'G':
                
                sl = df['low'] - (candle_size/100)
            
            elif df['candle'] == 'R':
                
                sl = df['high'] + (candle_size/100)
                
            return sl
        
        else:
            return 0
        
    
    def check_rsi(self, df):
        '''To check RSI and EMAs'''
        
#         ONLY TRADE WHEN 20 EMA IN FAVOUR

        if (df['candle'] == 'G' and df['RSI'] <= 68 and df['close'] > df['EMA_200'] and 
            df['trend'] == 'up' and df['EMA_20'] > df['EMA_200'] ):
            
                return 1
            
            
        elif (df['candle'] == 'R' and df['RSI'] >= 32 and df['close'] < df['EMA_200'] and 
              df['trend'] == 'down' and df['EMA_20'] < df['EMA_200'] ):
        
                return 1
        
 
        else:
            
            return 0
            
            
# To mark trades as Buy/ Sell

    def check_trade(self, df):
        
        if df['candle'] == 'G':
            
            return 'B'
        
        else:
            
            return 'S'
        
#  Main Calc method

    def mark_emas(self):
        '''Marking candles with colour'''
        
        self.df['candle'] = self.df.apply(self.candle, axis=1)
        
        '''Marking candles on emas'''
        
        self.df['pos_trade'] = self.df.apply(lambda x: '1' if  (
                               
                                x['high'] < x['upper_keltner'] and x['low'] > x['lower_keltner']
            
                                )
                                else '0', axis=1)
        

#     calculate climax with
#     RSI OB/OS and candle body outside keltner's channel extreme
        
        count_candles = 0
        
#       
        
    
#     For counting 20 candles after climax

        buy_flag = True
        sell_flag = True
        green_count = 0
        red_count = 0
        
        self.df['tradable'] = 0
        
        for index, row in self.df.iterrows():
            
            one_percent = row['open']/100 
            
#             When green candle goes outside channel
            if ( ((row['candle'] == 'G' and row['open'] > row['upper_keltner']) or
                
                (row['candle'] == 'R' and row['close'] > row['upper_keltner'])) and
                
                sell_flag == True and 
                
                ( abs(row['close'] - row['upper_keltner'])/one_percent ) >= 0.07 ):
        
                
                buy_flag = False
                green_count = 0
                self.df.at[index,'tradable'] = 'First G Break'
                
#             When price remain outside after green breakout
            elif( row['close'] > row['upper_keltner'] and sell_flag == True and 
                 buy_flag == False and green_count == 0):
                
                self.df.at[index,'tradable'] = 'Outside after G Break'
                

#            Any candle in 20 count closes outside channel, restart the count

            elif row['close'] > row['upper_keltner'] and buy_flag == True and green_count > 0:
                
                green_count = 0
                buy_flag = False
                self.df.at[index,'tradable'] = 'Outside after G Break'
            
#             When price returns inside channel after buy breakout
            elif row['close'] < row['upper_keltner'] and buy_flag == False:
                
                buy_flag = True
                green_count = 1
                self.df.at[index,'tradable'] = 'G count 1'
                
#                 Counting 20 candles when green breakout moves inside channel
            elif green_count > 0 and green_count < 19 and row['close'] < row['upper_keltner']:
                
                green_count += 1
                self.df.at[index,'tradable'] = 'Counting after G Break'
               
#             When 20 candles after climax are over
            elif green_count == 19 and row['close'] < row['upper_keltner'] and red_count == 0:
                
                green_count = 0
                self.df.at[index,'tradable'] = 0
                
                
#                 =============================================
                
#             When red candle goes outside channel
            if ( ((row['candle'] == 'R' and row['open'] < row['lower_keltner'] ) or
                 
                (row['candle'] == 'G' and row['close'] < row['lower_keltner']) )and 
                
                buy_flag == True  and
                
                ( abs(row['lower_keltner'] - row['close'])/one_percent ) >= 0.07 ):
                
                sell_flag = False
                red_count = 0
                self.df.at[index,'tradable'] = 'Red Breakout'
                
#             When price remain outside channel after red breakout

            elif row['close'] < row['lower_keltner'] and buy_flag == True and sell_flag == False:
                
                self.df.at[index,'tradable'] = 'Outside after Red Breakout'
                
                
#            Any candle in 20 count closes outside channel, restart the count

            elif row['close'] < row['lower_keltner'] and sell_flag == True and red_count > 0:
                
                red_count = 0
                sell_flag = False
                self.df.at[index,'tradable'] = 'Outside after R Break'
                

#             When price returns inside channel after sell breakout
            elif row['close'] > row['lower_keltner'] and sell_flag == False:
                
                sell_flag = True
                red_count = 1
                self.df.at[index,'tradable'] = 'Sell count 1'
                
#                 Counting 20 candles when red breakout moves inside channel
            elif red_count > 0 and red_count < 19 and row['close'] > row['lower_keltner']:
                
                red_count += 1
                self.df.at[index,'tradable'] = 'Counting after sell breakout'
               
#             When 20 candles after climax are over
            elif red_count == 19 and row['close'] > row['lower_keltner'] and green_count == 0:
                
                red_count = 0
                self.df.at[index,'tradable'] = 0
        
        
        
        

                
#             Calculating SL
        
        self.df['sl'] = self.df.apply(self.set_sl, axis=1)
        
#         Check trend
        self.squeeze = self.df.loc[(self.df['squeeze_on'] == False) & (self.df['pos_trade'] == '1')  & (self.df['trend'] != 'no trend')]
#         self.squeeze = self.df.loc[(self.df['squeeze_on'] == False) & (self.df['pos_trade'] == '1')]
        
        #          Check candle size
        self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)

        self.squeeze = self.squeeze.loc[self.squeeze['candle_size'] == 1]
        
        #  Check with prev candle
        self.squeeze['prev_candle'] = self.squeeze.apply(self.check_prev_candle, axis=1)
        
        self.squeeze = self.squeeze.loc[self.squeeze['prev_candle'] == 1]
        
        
#         Check RSI & EMAs
        self.squeeze['check_rsi'] = self.squeeze.apply(self.check_rsi, axis=1)
    
        self.squeeze = self.squeeze.loc[self.squeeze['check_rsi'] == 1]
        
#         mark trades as Buy or Sell
        self.squeeze['trade'] = self.squeeze.apply(self.check_trade, axis=1)
    
      
#         Check Climax
        self.squeeze = self.squeeze.loc[self.squeeze['tradable'] == 0]
        
        
        
    
    def pip_calc(self, close, sl):
#         Calculate pips between Entry & SL
        
        if str(close).index('.') >= 3:  # JPY pair
            multiplier = 0.01
        
        else:
            multiplier = 0.0001

        pips = round(abs(sl - close) / multiplier)
        return int(pips)
    
    
    def result_pip_calc(self, close, value):
#         Calculate pip value from currency price 
        
        if str(close).index('.') >= 3:  # JPY pair
            multiplier = 100
        
        else:
            multiplier = 10000
            
        pip = value * multiplier
        
        return float(pip)
        
    
    def calc_quantity(self, pips):
#         To find the number of lots to trade

        lots = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8,
                1.9, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        
#       Risk per trade 2%  of 10K $ account
        rpt = 200
    
#         print('Pips in calc_quantity: ', pips)

#         sl = 76

        quantity = []

        for i in lots:

            if (i * pips * self.one_pip_value) <= rpt:
                quantity.append(i)
        
        if quantity:

            return max(quantity)
        
        else:
            return 0

    
    def calc_exit(self, index : int, count):
        '''Calculating Exit, TRG and P/L'''

        sl = round(self.df.loc[index]['sl'], 3)
        f = 1
        trail_sl = False

        base_candle_index = index
   

        self.doji_count = 0
    
        pips = self.pip_calc(self.df.loc[index]['close'], self.df.loc[index]['sl'])
        
#         print('Pips: ', pips)
        quantity = self.calc_quantity(pips)
        
        self.result.loc[count,'QUANTITY'] = quantity
        
        candles_size = abs(self.df.loc[index]['high'] - self.df.loc[index]['low'])
        
            # Target 1:1.5
        if self.result.loc[count, 'TRADE'] == 'B':
            target = round(self.df.loc[index]['close'] + 1.5 * (self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
#             breakeven = round((self.df.loc[index]['close'] + target)/2, 3)
            breakeven = round(self.df.loc[index]['close'] + candles_size)
#             breakeven = round(self.df.loc[index]['close'] + 1.5 * (self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
    #         print('Trg: ', target)

        else:
            target = round(self.df.loc[index]['close'] - 1.5 * abs(self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
#             breakeven = round((self.df.loc[index]['close'] + target)/2, 3)
            breakeven = round(self.df.loc[index]['close'] - candles_size)
#             breakeven = round(self.df.loc[index]['close'] - 1.5 * abs(self.df.loc[index]['close'] - self.df.loc[index]['sl']), 3)
        
        # Target 1:2
        

        doji_counter = 0
        doji_flag = False
        
        # chart 0.04 % for doji
        chart = self.df.loc[index]['open']/100 * 0.04
        
#         For Buy trades

        if self.result.loc[count, 'TRADE'] == 'B':
        
            while f == 1:

                if index >= (len(self.df)-1):
                    break

                prev_candle_size = abs(self.df.loc[index]['close'] - self.df.loc[index]['open'])

                # First encounter with doji
                if (prev_candle_size <= chart) and (doji_flag == False):
                    doji_counter = 1
                    doji_flag = True
                    upper_limit = self.df.loc[index]['high']
                    lower_limit = self.df.loc[index]['low']

    #           Current candle check
                index += 1

                #  For Doji Check
                current_candle_high = self.df.loc[index]['high']
                current_candle_low = self.df.loc[index]['low']

                # if next is also doji
                if (doji_flag == True) and (current_candle_high <= upper_limit and current_candle_low >= lower_limit):
                    # Exit on 3 doji's in a row
                    if doji_counter > 2:
                        self.result.loc[count, 'EXIT'] = round(self.df.loc[index]['close'], 3)
                        self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
                        self.result.loc[count, 'REASON'] = 'DOJI EXIT'
                        f = 0
                    doji_counter += 1

                # For a non doji candle after doji
                else:
                    doji_flag = False
                    doji_counter = 0
                    upper_limit = None
                    lower_limit = None

                
                current_candle = self.df.loc[index]['candle']
                
                if current_candle == 'G':
                    # TRG  and Trail SL condition

                    if self.df.loc[index]['high'] >= target:
                        # TRG

                        self.result.loc[count, 'EXIT'] = target
                        self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
                        self.result.loc[count, 'REASON'] = 'TRG'
        #                 print('Exit: ', result.loc[count]['EXIT'])
                        f = 0

                    # For Breakeven
                    elif self.df.loc[index]['high'] >= breakeven:
                        sl = round(min(self.df.loc[base_candle_index]['close'], self.df.loc[base_candle_index]['EMA_8']), 3)
                        trail_sl = True


    #               For Gap Down exit
                    elif sl >= self.df.loc[index]['open']:

                        self.result.loc[count, 'EXIT'] =  round(self.df.loc[index]['open'], 3)
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0

                        self.result.loc[count, 'REASON'] = 'SL'
                        f = 0

                    # Green candle low hits SL 
                    elif sl >= self.df.loc[index]['low']:

                        self.result.loc[count, 'EXIT'] =  sl
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0

                        self.result.loc[count, 'REASON'] = 'SL'
                        f = 0

                elif current_candle == 'R':
                # SL condition

    #                 For Gap down
                    if sl >= self.df.loc[index]['open']:

                        self.result.loc[count, 'EXIT'] =  round(self.df.loc[index]['open'], 3)
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0
                        self.result.loc[count, 'REASON'] = 'SL'
                        f = 0

    #                 For SL
                    elif sl >= self.df.loc[index]['low']:

                        self.result.loc[count, 'EXIT'] = sl
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0
                        self.result.loc[count, 'REASON'] = 'SL'
        #                 print('Exit: ', result.loc[count]['EXIT'])
                        f = 0

            #  Max wait period 11 Days till breakeven 
            # or current candle closed above 1% to last
            # else exit the trade
                prev_close_one_percent = (self.df.loc[index - 1]['close'])/100

                if (index > (base_candle_index + 10)) and (self.df.loc[index]['close'] < breakeven) and (self.df.loc[index]['close'] <  (self.df.loc[index - 1]['close']) + prev_close_one_percent):
                    # print('------------------------------------------------')
                    # print('LATE: ', index)
                    # print('--------------------------------------------------------')
                    self.result.loc[count, 'EXIT'] = round(self.df.loc[index]['close'], 3)
                    self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
                    self.result.loc[count, 'REASON'] = 'NO MOVE'
                    f= 0
                
        elif self.result.loc[count, 'TRADE'] == 'S':
            
            while f == 1:

                if index >= (len(self.df)-1):
                    break

                prev_candle_size = abs(self.df.loc[index]['close'] - self.df.loc[index]['open'])

                # First encounter with doji
                if (prev_candle_size <= chart) and (doji_flag == False):
                    doji_counter = 1
                    doji_flag = True
                    upper_limit = self.df.loc[index]['high']
                    lower_limit = self.df.loc[index]['low']

    #           Current candle check
                index += 1

                #  For Doji Check
                current_candle_high = self.df.loc[index]['high']
                current_candle_low = self.df.loc[index]['low']

                # if next is also doji
                if (doji_flag == True) and (current_candle_high <= upper_limit and current_candle_low >= lower_limit):
                    # Exit on 3 doji's in a row
                    if doji_counter > 2:
                        self.result.loc[count, 'EXIT'] = round(self.df.loc[index]['close'], 3)
                        self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
                        self.result.loc[count, 'REASON'] = 'DOJI EXIT'
                        f = 0
                    doji_counter += 1

                # For a non doji candle after doji
                else:
                    doji_flag = False
                    doji_counter = 0
                    upper_limit = None
                    lower_limit = None

                
                current_candle = self.df.loc[index]['candle']
                
                if current_candle == 'R':
                    # TRG  and Trail SL condition

                    if self.df.loc[index]['low'] <= target:
                        # TRG

                        self.result.loc[count, 'EXIT'] = target
                        self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
                        self.result.loc[count, 'REASON'] = 'TRG'
        #                 print('Exit: ', result.loc[count]['EXIT'])
                        f = 0

                    # For Breakeven
                    elif self.df.loc[index]['low'] <= breakeven:
                        sl = round(min(self.df.loc[base_candle_index]['close'], self.df.loc[index]['EMA_8']), 3)
                        trail_sl = True


    #               For Gap up exit
                    elif sl <= self.df.loc[index]['open']:

                        self.result.loc[count, 'EXIT'] =  round(self.df.loc[index]['open'], 3)
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0

                        self.result.loc[count, 'REASON'] = 'SL'
                        f = 0

                    # Red candle High hits SL 
                    elif sl <= self.df.loc[index]['high']:

                        self.result.loc[count, 'EXIT'] =  sl
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0

                        self.result.loc[count, 'REASON'] = 'SL'
                        f = 0

                elif current_candle == 'G':
                # SL condition

    #                 For Gap UP
                    if sl <= self.df.loc[index]['open']:

                        self.result.loc[count, 'EXIT'] =  round(self.df.loc[index]['open'], 3)
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0
                        self.result.loc[count, 'REASON'] = 'SL'
                        f = 0

    #                 For SL
                    elif sl <= self.df.loc[index]['high']:

                        self.result.loc[count, 'EXIT'] = sl
                        self.result.loc[count, 'EXIT DATE'] = self.df.loc[index]['date']
                        if trail_sl:
                            self.result.loc[count, 'REASON'] = 'TRAIL_SL'
                            f = 0
                        self.result.loc[count, 'REASON'] = 'SL'
        #                 print('Exit: ', result.loc[count]['EXIT'])
                        f = 0

            #  Max wait period 11 Days till breakeven 
            # or current candle closed above 1% to last
            # else exit the trade
                prev_close_one_percent = (self.df.loc[index - 1]['close'])/100

                if (index > (base_candle_index + 10)) and (self.df.loc[index]['close'] > breakeven) and (self.df.loc[index]['close'] >  (self.df.loc[index - 1]['close']) - prev_close_one_percent):
                    # print('------------------------------------------------')
                    # print('LATE: ', index)
                    # print('--------------------------------------------------------')
                    self.result.loc[count, 'EXIT'] = round(self.df.loc[index]['close'], 3)
                    self.result.loc[count ,'EXIT DATE'] = self.df.loc[index]['date']
                    self.result.loc[count, 'REASON'] = 'NO MOVE'
                    f= 0
            
        
        if index < len(self.df):
            
            if self.result.loc[count, 'TRADE'] == 'B':

                self.result.loc[count, 'P/L'] = self.result.loc[count]['EXIT'] - self.result.loc[count]['ENTRY']
            
            else:
                
                 self.result.loc[count, 'P/L'] = self.result.loc[count]['ENTRY'] - self.result.loc[count]['EXIT']

            
            self.result.loc[count, 'REAL P/L'] = quantity * self.one_pip_value *  self.result_pip_calc(self.result.loc[count]['ENTRY'], self.result.loc[count, 'P/L'])
            
    
    
    def record_trades(self):
        '''Recording Trades in a csv
        Without overlapping trades'''

        self.result = pd.DataFrame(columns=['ENTRY DATE', 'ENTRY', 'QUANTITY', 'TRADE', 'EXIT', 'EXIT DATE', 'P/L', 'REAL P/L', 'S/L' ])

        self.result["EXIT DATE"] = pd.to_datetime(self.result["EXIT DATE"]) 

        for i in self.squeeze.index:
            count = len(self.result)

            #     print(position.loc[i]['time'])


            if count == 0 or (self.squeeze.loc[i]['date'] > self.result.loc[count-1]['EXIT DATE']):

            #     print(position.loc[i,'time'])
                self.result.loc[count, 'ENTRY DATE'] = self.squeeze.loc[i,'date']
            #     print('Time: ', position.loc[i]['time'])

                
#                 if self.squeeze.loc[i]['candle'] == 'G':
#                     self.result.loc[count, 'TRADE'] = 'B'
                    
#                 else:
#                     self.result.loc[count, 'TRADE'] = 'S'

                self.result.loc[count, 'TRADE'] = self.squeeze.loc[i]['trade']
                #     print('Position: ',  position.loc[i]['pos_trade'])

                self.result.loc[count, 'ENTRY'] = self.squeeze.loc[i]['close'] 
            #     print('Close: ', position.loc[i]['close'])

                self.result.loc[count, 'S/L'] = self.squeeze.loc[i]['sl']
            #     print('SL: ', position.loc[i]['sl'])

                # for Exit & P/L 
                self.calc_exit(i, count)
                
                
                
    def streak(self):
        '''For max profit and loss streak'''

        positive_counter = 0 
        negative_counter = 0
        max_score = 1
        min_score = 1

        for ele in self.result['P/L']:
            if ele > 0:

                if(negative_counter > 0):
                    min_score = max(min_score, negative_counter)

                positive_counter += 1
                negative_counter = 0

            elif ele < 0:

                if(positive_counter > 0):
                    max_score = max(max_score, positive_counter)

                positive_counter = 0
                negative_counter += 1 


        self.analyse.loc[0,'WINNING STREAK'] = max_score
        self.analyse.loc[0,'LOSING STREAK']= min_score

        self.analyse.loc[0, 'MAX DRAWDOWN'] = str(float(2 * self.analyse.loc[0,'LOSING STREAK']) ) + '% [2% Risk/Trade]'
    
    def calc_avg(self):
    
        '''Avg gain and loss'''

        self.analyse.loc[0,'AVG GAIN'] = round(float(self.result.loc[self.result['REAL P/L'] > 0]['REAL P/L'].sum() /  self.analyse['WINS']), 2)

        self.analyse.loc[0,'AVG LOSS'] = round(float(self.result.loc[self.result['REAL P/L'] < 0]['REAL P/L'].sum() / self.analyse['LOSSES']), 2)

        
    def calc_result(self):
        '''Make Stats'''
        
        self.analyse.loc[0,'TOTAL TRADES'] = len(self.result)
        self.analyse.loc[0,'WINS'] = int(len(self.result.loc[self.result['P/L'] > 0]))
        self.analyse.loc[0,'LOSSES'] = int(len(self.result.loc[self.result['P/L'] < 0]))
        # Performance :- ((P/L)/ ENTRY) * 100
        # self.analyse.loc[0,'PERFORMANCE'] = str(float(( 4 * self.analyse.loc[0,'WINS']) - (2 * self.analyse.loc[0,'LOSSES']))) + '%'
        self.analyse.loc[0,'WIN RATE'] = str(round(float((self.analyse['WINS'] / self.analyse['TOTAL TRADES']) * 100), 2)) + '%'
        performance = round((self.result['REAL P/L'].sum() / self.account) * 100, 2)
        self.analyse.loc[0,'PERFORMANCE'] = str( performance ) + ' %' 

        '''Avg gain and loss'''
        self.calc_avg()

        '''Max streaks calculation'''
        self.streak()

        #   Calc performance
        # self.performance()

#         print(self.analyse)

        


In [11]:
currency = BackTest(['GBPAUDm',6.403, 24000])
currency.indicators()
currency.mark_emas()
currency.record_trades()
currency.calc_result()
currency.display()

Working on:  GBPAUDm


C:\Users\hp\AppData\Local\Temp\ipykernel_3136\570890219.py:544: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)


  TOTAL TRADES WINS LOSSES PERFORMANCE WIN RATE WINNING STREAK LOSING STREAK  \
0          382  196    186     72.42 %   51.31%             20            11   

            MAX DRAWDOWN AVG GAIN AVG LOSS  
0  22.0% [2% Risk/Trade]   172.67  -143.02  


In [3]:
total = pd.DataFrame(columns=[ 'STOCK','TOTAL TRADES', 'WINS', 'LOSSES', 'PERFORMANCE', 'WIN RATE', 
                                'WINNING STREAK', 'LOSING STREAK', 'MAX DRAWDOWN', 'AVG GAIN', 'AVG LOSS'])

In [9]:
pair_list = [['EURUSDm',10, 24000],['GBPUSDm', 10, 24000],['GBPAUDm',6.403, 24000],['USDCADm',7.3511, 24000],['GBPCHFm',11.30608, 24000],['EURGBPm',12.5764, 24000 ],
             ['USDCHFm',11.30608, 24000],['NZDUSDm',10, 24000], ['GBPNZDm', 5.9008, 24000],  ['AUDUSDm',10, 24000], ['EURAUDm',6.403, 24000], ['GBPCADm',7.3511, 24000], 
             ['AUDNZDm',5.9008, 24000],['AUDCADm',7.3511, 24000],['NZDCADm', 7.3511, 24000],['CADCHFm', 11.30608, 24000],['AUDCHFm',11.30608, 24000]
             ,['NZDCHFm', 11.30608, 24000], ['USDJPYm', 6.82682, 24000],['AUDJPYm',6.82682,24000],['CADJPYm',6.82682,24000],['GBPJPYm',6.82682,24000],
             ['EURJPYm',6.82682,24000]]

#  Logic is not correct for them
# pair_list = [['USDJPYm', 6.82682, 24000],['AUDJPYm',6.82682,24000],['CADJPYm',6.82682,24000],['GBPJPYm',6.82682,24000],
#              ['EURJPYm',6.82682,24000]]


count=0
for i in pair_list:
    currency = BackTest(i)
    currency.indicators()
    currency.mark_emas()
    currency.record_trades()
    currency.calc_result()
    analyse = currency.display()
    total.loc[count, 'STOCK'] = i
    total.loc[count, 'TOTAL TRADES']  = analyse['TOTAL TRADES'][0]
    total.loc[count, 'WINS'] = analyse['WINS'][0]
    total.loc[count, 'LOSSES'] = analyse['LOSSES'][0]
    total.loc[count, 'PERFORMANCE'] = analyse['PERFORMANCE'][0]
    total.loc[count, 'WIN RATE'] = analyse['WIN RATE'][0]
    total.loc[count, 'WINNING STREAK'] = analyse['WINNING STREAK'][0]
    total.loc[count, 'LOSING STREAK'] = analyse['LOSING STREAK'][0]
    total.loc[count, 'MAX DRAWDOWN'] = analyse['MAX DRAWDOWN'][0]
    total.loc[count, 'AVG GAIN'] = analyse['AVG GAIN'][0]
    total.loc[count, 'AVG LOSS'] = analyse['AVG LOSS'][0]
    
    count += 1
    
total.to_excel('Forex Total Result no ema update.xlsx')

Working on:  EURUSDm


C:\Users\hp\AppData\Local\Temp\ipykernel_10420\362870147.py:542: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)


Working on:  GBPUSDm


C:\Users\hp\AppData\Local\Temp\ipykernel_10420\362870147.py:542: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)


Working on:  GBPAUDm


...

C:\Users\hp\AppData\Local\Temp\ipykernel_10420\362870147.py:542: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)


Working on:  GBPJPYm


C:\Users\hp\AppData\Local\Temp\ipykernel_10420\362870147.py:542: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)


Working on:  EURJPYm


C:\Users\hp\AppData\Local\Temp\ipykernel_10420\362870147.py:542: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)


In [26]:
# JPY pairs code is not correct

total.to_excel('Forex Total Result emas.xlsx')

In [5]:
currency = BackTest(['EURUSDm',10, 24000])
currency.indicators()
currency.mark_emas()
currency.record_trades()
currency.calc_result()
currency.display()

Working on:  EURUSDm


C:\Users\hp\AppData\Local\Temp\ipykernel_3136\570890219.py:544: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.squeeze['candle_size'] = self.squeeze.apply(self.check_body, axis=1)


  TOTAL TRADES WINS LOSSES PERFORMANCE WIN RATE WINNING STREAK LOSING STREAK  \
0          221   88    133     12.45 %   39.82%              7             6   

            MAX DRAWDOWN AVG GAIN AVG LOSS  
0  12.0% [2% Risk/Trade]   197.38  -121.24  


In [ ]:
# FIXED 200 EMA ENTRY IN WRONG DIRECTION